https://assaeunji.github.io/machine%20learning/2020-11-29-implicitfeedback/

# test 3주 정합성 검증
- 

# 추천 모델
- ALS MF, LMF, MP (총 3개)
- 총 3개의 추천을 진행하며 MF와 LMF 의 경우 콜드스타트 유저(신규 유저)인 경우 MP로 추천 진행

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
import implicit
from implicit.als import AlternatingLeastSquares as ALS

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import scipy.sparse as sp
from implicit.lmf import LogisticMatrixFactorization as LMF

/home/user_3/medistream-recsys/Script


# 1.Dataload

In [5]:
# products name 확인 용
products_df = pd.read_json("/fastcampus-data/products/products.json")
products_df = key_to_element(['_id'],products_df)

100%|██████████| 5141/5141 [00:00<00:00, 655587.15it/s]


In [6]:
df = pd.read_json('/fastcampus-data/select_column_version_4.json')

In [7]:
df['date_paid'] = pd.to_datetime(df['date_paid'])

In [8]:
df['date_paid'].max()

Timestamp('2022-09-13 08:59:21.151000+0000', tz='UTC')

In [9]:
split_day_list =  []
test_day_list = []

# 1일 test 날짜 확인
last_date_state = '2022-09-13'
split_day = pd.to_datetime(last_date_state)-relativedelta(months=2)
test_day = pd.to_datetime(last_date_state)
'''
마지막 날짜에서 개월 수를 자름 -> split_day
months 만 바꾸면 21번 분량이 나옴
'''

for i in range(0,21):
    sp_day = str((split_day-timedelta(days=i)).to_pydatetime().date())
    tt_day  = str((test_day-timedelta(days=i)).to_pydatetime().date())
    split_day_list.append(sp_day)
    test_day_list.append(tt_day)

In [10]:
class CustomEvaluator:
    # relavence 모두 1로 동일하게 봄
    def _idcg(self, l):
        return sum((1.0 / np.log(i + 2) for i in range(l)))
    

    def __init__(self):
        self._idcgs = [self._idcg(i) for i in range(1000)]
    '''
    idcgs 예시, item 3개 추천되므로 3.074281787960283 가 됩니다.
    [0, 1.4426950408889634, 2.352934267515801, 3.074281787960283]
    '''

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)

        return dcg / self._idcgs[len(gt)]
    
    def _entropy_diversity(self,rec_list):
        import six
        import math
        
        topn = len(rec_list[0]['items'])
        users = [i.get('id',None) for i in rec_list]
        sz = float(len(users)) * topn
        freq = {}
        for rec in rec_list:
            for r in rec['items']:
                freq[r] = freq.get(r, 0) + 1
        ent = -sum([v / sz * math.log(v / sz) for v in six.itervalues(freq)])
        return ent

    def _eval(self, gt_list, rec_list):
        gt_dict = {g["id"]: g for g in gt_list}
        ndcg_score = 0.0

        for rec in rec_list:
            gt = gt_dict[rec["id"]]
            ndcg_score += self._ndcg(gt["items"], rec["items"])


        ndcg_score = ndcg_score / len(rec_list)
        ent = self._entropy_diversity(rec_list)
        
        return ndcg_score, ent

    def evaluate(self, gt_list, rec_list):
        try:
            ndcg_score, ent_score = self._eval(gt_list, rec_list)
            print(f"NDCG: {ndcg_score:.6}")
            print(f"Entropy Diversity: {ent_score:.6} ")
        except Exception as e:
            print(e)


In [11]:
def module(df:pd.DataFrame(), split_date, test_date)->pd.DataFrame():
    
    # paid orders만 가져오기
    df['date_paid'] = pd.to_datetime(df['date_paid'])
    df_only_paid = df[~df['date_paid'].isna()]
    # 3개월치 데이터만 가져오기
    df_date = df_only_paid[df_only_paid['date_paid'] >= split_date]
    # 취소 안된 것만 가져오기
    complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
    # 도서 카테고리만 가져오기
    only_book = complete_df[complete_df['name'] == '도서']

    # 유저가 중복으로 아이템 구매 삭제
    df_duplicated_book = only_book.drop_duplicates(subset=['customer_id','product_ids'])
    df_book = df_duplicated_book.sort_values(by='date_paid').reset_index(drop=True)
    
    train = df_book[df_book['date_paid'] < test_date]
    test = df_book[df_book['date_paid'].dt.date == pd.to_datetime(test_date)]
    
    
    # test 만 있는 item 제거
    only_test_items = set(test.product_ids.unique())-set(train.product_ids.unique())
    if_prepro_test = test[~test['product_ids'].isin(only_test_items)]
    
    
    PdIds = train.product_ids.unique()

    PdIdToIndex = {}
    indexToPdId = {}

    colIdx = 0

    for PdId in PdIds:
        PdIdToIndex[PdId] = colIdx
        indexToPdId[colIdx] = PdId
        colIdx += 1

    userIds = train.customer_id.unique()

    userIdToIndex = {}
    indexToUserId = {}

    rowIdx = 0

    for userId in userIds:
        userIdToIndex[userId] = rowIdx
        indexToUserId[rowIdx] = userId
        rowIdx += 1
    rows = []
    cols = []
    vals = []

    for row in train.itertuples():
        rows.append(userIdToIndex[row.customer_id])
        cols.append(PdIdToIndex[row.product_ids])
        vals.append(1)

    purchase_sparse = sp.csr_matrix((vals, (rows, cols)), shape=(rowIdx,colIdx))

    matrix = purchase_sparse.todense()

    medistream_prediction_df = train[['date_created','regular_price','sale_price','three_months','product_ids','name_x']]
    medistream_prediction_preprop_df = medistream_prediction_df.drop_duplicates(subset=['product_ids'], ignore_index=True)
    medistream_prediction_preprop_df['date_created'] = pd.to_datetime(medistream_prediction_preprop_df['date_created'])    
    
    als_model = ALS(factors=20,regularization=0.01, iterations = 50, random_state=42)
    alsmodel = als_model.fit(purchase_sparse)
    lmf_model = LMF(factors=20, regularization=0.001, iterations = 20, random_state=42)
    lmfmodel = lmf_model.fit(purchase_sparse)
    
    most_popular = train.groupby(['product_ids','name_x']).count()['customer_id'].reset_index()
    most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index
    

        # test 예측값, 이미 구매 했을 경우 제외
    als_predict_list = []
    for user_id in test['customer_id'].unique():
        try:
            result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
            als_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
        except:
            train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
            als_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                ]})

        # 100 개만 예측하기
    for idx, pred_list in enumerate(als_predict_list):
        als_predict_list[idx]['items'] = pred_list['items'][:100]

    

        # test 예측값
    lmf_predict_list = []
    for user_id in test['customer_id'].unique():
        try:
            result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
            lmf_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
        except:
            train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
            lmf_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                ]})

        # 100 개만 예측하기
    for idx, pred_list in enumerate(lmf_predict_list):
        lmf_predict_list[idx]['items'] = pred_list['items'][:100]

    

        # test 예측값, 이미 구매 했을 경우 제외
    predict_popular_list = []
    for user_id in test['customer_id'].unique():
        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
        predict_popular_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                ]})

        # 100 개만 예측하기
    for idx, pred_list in enumerate(predict_popular_list):
        predict_popular_list[idx]['items'] = pred_list['items'][:100]
        
    # real test 
    ground_trues = []
    for user_id in test['customer_id'].unique():
        ground_trues.append({'id': user_id,\
        'items':list(test[test['customer_id']==user_id].product_ids)
        })
    # ALS 
    evaluator = CustomEvaluator()
    als = evaluator._eval(ground_trues, als_predict_list)
    
    # LMF 
    evaluator = CustomEvaluator()
    lmf = evaluator._eval(ground_trues, lmf_predict_list)
    # MP
    evaluator = CustomEvaluator()
    mp = evaluator._eval(ground_trues, predict_popular_list)
    
    # 인기도순
    medistream_popular_list = medistream_prediction_preprop_df.sort_values(by='three_months', ascending=False).index
    # 최신순
    medistream_latest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=False).index
    # 오랜된 순
    medistream_oldest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=True).index
    # 높은 가격 순
    medistream_high_price_list = medistream_prediction_preprop_df.sort_values(by='sale_price', ascending=False).index
    # 낮은 가격 순
    medistream_low_price_list = medistream_prediction_preprop_df.sort_values(by='sale_price', ascending=True).index
    # 이름 순
    medistream_name_sort_list = medistream_prediction_preprop_df.sort_values(by='name_x',ascending=True).index

    def medistream_prediction_method(predict_num:int ,medi_predict_list:list)->list:
        medistream_predict_list = []
        for user_id in test['customer_id'].unique():
            medistream_predict_list.append({'id':user_id ,'items':[medistream_prediction_preprop_df.product_ids.loc[num] \
                                                                           for num in medi_predict_list]})

        # 100 개만 예측하기
        for idx, pred_list in enumerate(medistream_predict_list):
            medistream_predict_list[idx]['items'] = pred_list['items'][:predict_num]

        return medistream_predict_list
    
    medistream_predict_popular_list = medistream_prediction_method(100, medistream_popular_list)
    medistream_predict_latest_list = medistream_prediction_method(100, medistream_latest_list)
    medistream_predict_oldest_list = medistream_prediction_method(100, medistream_oldest_list)
    medistream_predict_high_price_list = medistream_prediction_method(100, medistream_high_price_list)
    medistream_predict_low_price_list = medistream_prediction_method(100, medistream_low_price_list)
    medistream_predict_name_sort_list = medistream_prediction_method(100, medistream_name_sort_list)
    
    def medistream_prediction(ground_trues:list, predict_list:list):
        evaluator = CustomEvaluator()
        ndcg, entropy = evaluator._eval(ground_trues, predict_list)

        assert len(predict_list) == len(ground_trues)

        cnt = 0
        for gt, pred_list in zip(ground_trues, predict_list):
            for pred in pred_list['items']:
                if pred in gt['items']:
                    cnt += 1
        return ndcg, entropy, cnt
    
    medistream_predict_score = {'medistream_predict':['medi_popular','latest','oldest','high_price','low_price','name_sort'], \
                                'ndcg':[], 'entropy':[], 'cnt':[]}

    medistream_predict_list = [medistream_predict_popular_list, medistream_predict_latest_list, medistream_predict_oldest_list,\
                              medistream_predict_high_price_list, medistream_predict_low_price_list, medistream_predict_name_sort_list]

    for medistream_predict in medistream_predict_list:
        ndcg, entropy, cnt = medistream_prediction(ground_trues, medistream_predict)
        medistream_predict_score['ndcg'].append(ndcg)
        medistream_predict_score['entropy'].append(entropy)
        medistream_predict_score['cnt'].append(cnt)

        
######## hyper parameter
    als_mf_hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[],'entropy':[]}

    factors = [5,10,15,20]
    regularizations = [0.01,0.005]
    iterations = [5,10,15,20,25,30,40,50]

    for factor in factors:
        for regularization in regularizations:
            for iteration in iterations:
                als_model = ALS(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
                als_model.fit(purchase_sparse, show_progress=False)

                # 신규 유저인 경우 mp로 넣기
                # 전체 도서에 대한 판매 만큼 정렬 후 넣기
                most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

                # test 예측값, 이미 구매 했을 경우 제외
                als_predict_list = []
                for user_id in test['customer_id'].unique():
                    try:
                        result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
                        als_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
                    except:
                        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
                        als_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                            ]})

                # 100 개만 예측하기
                for idx, pred_list in enumerate(als_predict_list):
                    als_predict_list[idx]['items'] = pred_list['items'][:100]

                # ALS 
                evaluator = CustomEvaluator()
                ndcg, entropy = evaluator._eval(ground_trues, als_predict_list)

                als_mf_hyper_parameter['factor'].append(factor)
                als_mf_hyper_parameter['regularization'].append(regularization)
                als_mf_hyper_parameter['iteration'].append(iteration)
                als_mf_hyper_parameter['NDCG'].append(ndcg)
                als_mf_hyper_parameter['entropy'].append(entropy)
                
                
    lmf_hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[],'entropy':[]}

    factors = [5,10,15,20]
    regularizations = [0.01,0.005]
    iterations = [5,10,15,20,25,30,40,50]

    for factor in factors:
        for regularization in regularizations:
            for iteration in iterations:
                lmf_model = LMF(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
                lmf_model.fit(purchase_sparse, show_progress=False)

                # 신규 유저 mp로 넣기
                most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

                # test 예측값
                lmf_predict_list = []
                for user_id in test['customer_id'].unique():
                    try:
                        result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
                        lmf_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
                    except:
                        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
                        lmf_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                            ]})

                # 100 개만 예측하기
                for idx, pred_list in enumerate(lmf_predict_list):
                    lmf_predict_list[idx]['items'] = pred_list['items'][:100]

                # LMF
                evaluator = CustomEvaluator()
                ndcg, entropy = evaluator._eval(ground_trues, lmf_predict_list)

                lmf_hyper_parameter['factor'].append(factor)
                lmf_hyper_parameter['regularization'].append(regularization)
                lmf_hyper_parameter['iteration'].append(iteration)
                lmf_hyper_parameter['NDCG'].append(ndcg)
                lmf_hyper_parameter['entropy'].append(entropy)
                
#     return als_mf_hyper_parameter, lmf_hyper_parameter, medistream_predict_score

    all_prediction_df = {'first_day':[],'last_day':[],'train_데이터수':[],'train_유저수':[],'test_데이터수':[],\
        'test_유저수':[],'test_신규유저수':[],'test_신규아이템수':[],'원본_test수':[],'전처리진행test수':[],\
        'als_mf':[],'lmf':[],'mp':[],'medi_popular':[],'latest':[],\
        'oldest':[],'high_price':[],'low_price':[],'name_sort':[],\
         'als_mf_entropy':[],'lmf_entropy':[],'mp_entropy':[],'medi_popular_entropy':[],'latest_entropy':[],\
         'oldest_entropy':[],'high_price_entropy':[],'low_price_entropy':[],'name_sort_entropy':[]}
    medistream_predict_df = pd.DataFrame(medistream_predict_score)

    all_prediction_df['first_day'].append(str(datetime.date(train['date_paid'].min()))+' '+str(datetime.date(train['date_paid'].max())))
    all_prediction_df['last_day'].append(str(datetime.date(test['date_paid'].min()))+' '+str(datetime.date(test['date_paid'].max())))
    all_prediction_df['train_데이터수'].append(len(train))
    all_prediction_df['train_유저수'].append(len(set(train.customer_id)))
    all_prediction_df['test_데이터수'].append(len(test))
    all_prediction_df['test_유저수'].append(len(set(test.customer_id)))
    all_prediction_df['test_신규유저수'].append(len(set(test['customer_id'].unique())- set(train['customer_id'].unique())))
    all_prediction_df['test_신규아이템수'].append(len(set(test.product_ids.unique())-set(train.product_ids.unique())))
    all_prediction_df['원본_test수'].append(len(test))
    all_prediction_df['전처리진행test수'].append(len(if_prepro_test))

    # ndcg
    all_prediction_df['als_mf'].append(pd.DataFrame(als_mf_hyper_parameter).sort_values(by='NDCG',ascending=False)['NDCG'].iloc[0])
    all_prediction_df['lmf'].append(pd.DataFrame(lmf_hyper_parameter).sort_values(by='NDCG',ascending=False)['NDCG'].iloc[0])
    all_prediction_df['mp'].append(evaluator._eval(ground_trues, predict_popular_list)[0])
    all_prediction_df['medi_popular'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='medi_popular'].iloc[0]['ndcg'])
    all_prediction_df['latest'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='latest'].iloc[0]['ndcg'])
    all_prediction_df['oldest'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='oldest'].iloc[0]['ndcg'])
    all_prediction_df['high_price'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='high_price'].iloc[0]['ndcg'])
    all_prediction_df['low_price'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='low_price'].iloc[0]['ndcg'])
    all_prediction_df['name_sort'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='name_sort'].iloc[0]['ndcg'])

#     entropy
    all_prediction_df['als_mf_entropy'].append(pd.DataFrame(als_mf_hyper_parameter).sort_values(by='entropy',ascending=False)['entropy'].iloc[0])
    all_prediction_df['lmf_entropy'].append(pd.DataFrame(lmf_hyper_parameter).sort_values(by='entropy',ascending=False)['entropy'].iloc[0])
    all_prediction_df['mp_entropy'].append(evaluator._eval(ground_trues, predict_popular_list)[1])
    all_prediction_df['medi_popular_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='medi_popular'].iloc[0]['entropy'])
    all_prediction_df['latest_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='latest'].iloc[0]['entropy'])
    all_prediction_df['oldest_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='oldest'].iloc[0]['entropy'])
    all_prediction_df['high_price_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='high_price'].iloc[0]['entropy'])
    all_prediction_df['low_price_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='low_price'].iloc[0]['entropy'])
    all_prediction_df['name_sort_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='name_sort'].iloc[0]['entropy'])

    print('train 총 기간:',train['date_paid'].max()-train['date_paid'].min())
    print('test 총 기간:',test['date_paid'].max()-test['date_paid'].min())
    
    return pd.DataFrame(all_prediction_df)

In [15]:
from tqdm import tqdm

twenty_df_list = []
for sp_day, tt_day in tqdm(zip(split_day_list,test_day_list)):
    module_df = module(df, sp_day,tt_day)
    twenty_df_list.append(module_df)
twenty_df = pd.concat(twenty_df_list, ignore_index=True)
twenty_df

0it [00:00, ?it/s]<ipython-input-11-b3ae7dbb3584>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medistream_prediction_preprop_df['date_created'] = pd.to_datetime(medistream_prediction_preprop_df['date_created'])


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

1it [03:51, 231.36s/it]

train 총 기간: 61 days 22:12:31.689000
test 총 기간: 0 days 08:44:11.628000


...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

22it [1:28:19, 240.89s/it]

train 총 기간: 61 days 12:00:12.527000
test 총 기간: 0 days 20:54:19.574000


,first_day,last_day,train_데이터수,train_유저수,test_데이터수,test_유저수,test_신규유저수,test_신규아이템수,원본_test수,전처리진행test수,als_mf,lmf,mp,medi_popular,latest,oldest,high_price,low_price,name_sort,als_mf_entropy,lmf_entropy,mp_entropy,medi_popular_entropy,latest_entropy,oldest_entropy,high_price_entropy,low_price_entropy,name_sort_entropy
0,2022-07-13 2022-09-12,2022-09-13 2022-09-13,3211,1864,16,13,6,1,16,15,0.300905,0.324448,0.225128,0.212715,0.137371,0.107744,0.085096,0.096623,0.149090,5.143272,5.111985,4.622626,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
1,2022-07-12 2022-09-11,2022-09-12 2022-09-12,3218,1868,15,11,6,0,15,15,0.303104,0.377956,0.296826,0.292180,0.073244,0.153792,0.148777,0.052820,0.116050,5.074641,5.073365,4.634161,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
2,2022-07-11 2022-09-10,2022-09-11 2022-09-11,3247,1876,4,2,1,1,4,3,0.470970,0.464996,0.415777,0.425172,0.131070,0.050361,0.000000,0.185373,0.111697,4.930949,4.917086,4.612102,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
3,2022-07-10 2022-09-09,2022-09-10 2022-09-10,3259,1881,2,1,0,0,2,2,0.405937,0.552549,0.242658,0.230722,0.000000,0.227222,0.109785,0.122629,0.102191,4.605170,4.605170,4.605170,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
4,2022-07-09 2022-09-08,2022-09-09 2022-09-09,3274,1883,6,4,1,0,6,6,0.311133,0.260073,0.163455,0.162349,0.025951,0.291830,0.163675,0.025951,0.024388,5.140645,5.101672,4.626355,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
5,2022-07-08 2022-09-07,2022-09-08 2022-09-08,3195,1843,99,81,49,1,99,98,0.332975,0.440913,0.432362,0.398089,0.407638,0.040662,0.139718,0.033646,0.033929,5.087791,5.058015,4.630888,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
6,2022-07-07 2022-09-06,2022-09-07 2022-09-07,3209,1847,33,22,10,2,33,31,0.272029,0.315825,0.324265,0.288425,0.149893,0.140753,0.088671,0.105318,0.069086,5.170550,5.151723,4.624952,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
7,2022-07-06 2022-09-05,2022-09-06 2022-09-06,3164,1836,58,50,16,0,58,58,0.112644,0.149212,0.112580,0.291670,0.084552,0.143201,0.140163,0.023735,0.126009,5.225954,5.206014,4.650333,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
8,2022-07-05 2022-09-04,2022-09-05 2022-09-05,3185,1838,25,19,13,0,25,25,0.315869,0.336995,0.290964,0.332526,0.208003,0.078518,0.093099,0.087342,0.069438,5.010331,5.002523,4.613856,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
9,2022-07-04 2022-09-03,2022-09-04 2022-09-04,3243,1860,20,15,7,0,20,20,0.270799,0.318355,0.270099,0.267212,0.132036,0.186405,0.166661,0.049126,0.112203,5.135857,5.139620,4.636215,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
